### Imports

In [1]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

### Load data (raw imdb dataset)

In [2]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2023-10-26 07:37:25--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  23.6MB/s    in 4.8s    

2023-10-26 07:37:31 (16.8 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [3]:
# Remove train/unsup subdirectory
!rm -r aclImdb/train/unsup

In [4]:
# Prepare validation data (50% of test data, so alltogether we have )
# Imports
import os, shutil, random

BASE_DIR_TEST = "aclImdb/test"
ALL_DIR_POS = os.listdir(BASE_DIR_TEST + '/pos')
ALL_DIR_NEG = os.listdir(BASE_DIR_TEST + '/neg')

# print(len(ALL_DIR_POS), len(ALL_DIR_NEG))

# Define number of validation samples
VAL_SAMPLES = int(0.5 * len(ALL_DIR_POS + ALL_DIR_NEG))

# Shuffle files
random.shuffle(ALL_DIR_POS)
random.shuffle(ALL_DIR_NEG)

# Pick appropriate number of validation files
val_paths_pos = ALL_DIR_POS[:VAL_SAMPLES//2] # half samples will be positive
val_paths_neg = ALL_DIR_NEG[:VAL_SAMPLES//2] # half of samples will be negative

# Create directories appropriate for validation files
os.makedirs('aclImdb/val/pos')
os.makedirs('aclImdb/val/neg')

# Move all validation files into val/pos and val/neg directories
for file in val_paths_pos:
  shutil.move(src = "aclImdb/test/pos/" + file, dst = "aclImdb/val/pos/" + file)

for file in val_paths_neg:
  shutil.move(src = "aclImdb/test/neg/" + file, dst = "aclImdb/val/neg/" + file)

### Create train, validation and test datasets using `text_dataset_from_directory`

In [5]:
BATCH_SIZE = 32

# Create train dataset
train_data = tf.keras.utils.text_dataset_from_directory("aclImdb/train", batch_size = BATCH_SIZE)

# Create validation dataset
val_data = tf.keras.utils.text_dataset_from_directory("aclImdb/val", batch_size = BATCH_SIZE)

# Create test dataset
test_data = tf.keras.utils.text_dataset_from_directory("aclImdb/test", batch_size = BATCH_SIZE)

Found 25000 files belonging to 2 classes.
Found 12500 files belonging to 2 classes.
Found 12500 files belonging to 2 classes.


# First attempt: bag of words model

### Preprocess data with TextVectorization layer

In [6]:
# Only consider single words (ngram = 1) and multi-hot encode each sentence
tv = tf.keras.layers.TextVectorization(max_tokens = 20000,
                                       ngrams = 1,
                                       output_mode = 'multi_hot')

# Extract all messages from training data (leave targets out)
train_data_text_only = train_data.map(lambda x, y: x)

# Build vocabulary from training data using TextVectorizer
tv.adapt(train_data_text_only)

# Prepare processed versions of datasets (text multi-hot encoded using vocabulary trained on training data)
train_data_preprocessed_1_gram = train_data.map(lambda x, y: (tv(x), y), num_parallel_calls = 4)
val_data_preprocessed_1_gram = val_data.map(lambda x, y: (tv(x), y), num_parallel_calls = 4)
test_data_preprocessed_1_gram = test_data.map(lambda x, y: (tv(x), y), num_parallel_calls = 4)


In [7]:
# Check if everything OK
for inputs, targets in train_data_preprocessed_1_gram:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
### Build and train simple dense classification model

def build_model(num_tokens = 20000, hidden_units = 16):
  inputs = tf.keras.layers.Input(shape = (num_tokens, ))
  x = tf.keras.layers.Dense(hidden_units, activation = 'relu')(inputs)
  x = tf.keras.layers.Dropout(rate = 0.5)(x)
  outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

  model = tf.keras.models.Model(inputs = inputs, outputs = outputs)

  model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

  return model

model = build_model()

history = model.fit(train_data_preprocessed_1_gram,
          epochs = 10,
          validation_data = val_data_preprocessed_1_gram)



Epoch 1/10
782/782 [==============================] - 15s 18ms/step - loss: 0.4000 - accuracy: 0.8357 - val_loss: 0.2919 - val_accuracy: 0.8882
Epoch 2/10
782/782 [==============================] - 12s 16ms/step - loss: 0.2426 - accuracy: 0.9133 - val_loss: 0.2812 - val_accuracy: 0.8854
Epoch 3/10
782/782 [==============================] - 12s 16ms/step - loss: 0.1790 - accuracy: 0.9384 - val_loss: 0.2954 - val_accuracy: 0.8822
Epoch 4/10
782/782 [==============================] - 13s 17ms/step - loss: 0.1388 - accuracy: 0.9512 - val_loss: 0.3273 - val_accuracy: 0.8817
Epoch 5/10
782/782 [==============================] - 12s 16ms/step - loss: 0.1101 - accuracy: 0.9634 - val_loss: 0.3619 - val_accuracy: 0.8805
Epoch 6/10
782/782 [==============================] - 13s 16ms/step - loss: 0.0920 - accuracy: 0.9689 - val_loss: 0.3932 - val_accuracy: 0.8782
Epoch 7/10
782/782 [==============================] - 13s 16ms/step - loss: 0.0792 - accuracy: 0.9730 - val_loss: 0.4432 - val_accuracy:

#### We achieve ~ 89% accuracy. Let's try the same with 2-grams and 3-grams

In [ ]:
# Build new TextVectorizers, one for 2-grams and one for 3-grams
tv2 = tf.keras.layers.TextVectorization(max_tokens = 20000,
                                     output_mode = 'multi_hot',
                                     ngrams = 2)

tv3 = tf.keras.layers.TextVectorization(max_tokens = 20000,
                                     output_mode = 'multi_hot',
                                     ngrams = 3)

# Train both both vectorizers on train data texsts
tv2.adapt(train_data_text_only)
tv3.adapt(train_data_text_only)

# Prepare preprocessed versions of all datasets
train_data_preprocessed_2_gram = train_data.map(lambda x, y: (tv2(x), y), num_parallel_calls = 4)
train_data_preprocessed_3_gram = train_data.map(lambda x, y: (tv3(x), y), num_parallel_calls = 4)

val_data_preprocessed_2_gram = val_data.map(lambda x, y: (tv2(x), y), num_parallel_calls = 4)
val_data_preprocessed_3_gram = val_data.map(lambda x, y: (tv3(x), y), num_parallel_calls = 4)

test_data_preprocessed_2_gram = test_data.map(lambda x, y: (tv2(x), y), num_parallel_calls = 4)
test_data_preprocessed_3_gram = test_data.map(lambda x, y: (tv3(x), y), num_parallel_calls = 4)

In [ ]:
# Build and train models
model2 = build_model()

history2 = model.fit(train_data_preprocessed_2_gram,
          epochs = 10,
          validation_data = val_data_preprocessed_2_gram)

model3 = build_model()

history3 = model.fit(train_data_preprocessed_3_gram,
          epochs = 10,
          validation_data = val_data_preprocessed_3_gram)

Epoch 1/10
782/782 [==============================] - 17s 20ms/step - loss: 0.6591 - accuracy: 0.7088 - val_loss: 0.4234 - val_accuracy: 0.8382
Epoch 2/10
782/782 [==============================] - 18s 23ms/step - loss: 0.4367 - accuracy: 0.8155 - val_loss: 0.3786 - val_accuracy: 0.8653
Epoch 3/10
782/782 [==============================] - 14s 18ms/step - loss: 0.3612 - accuracy: 0.8576 - val_loss: 0.3487 - val_accuracy: 0.8718
Epoch 4/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3045 - accuracy: 0.8832 - val_loss: 0.3555 - val_accuracy: 0.8742
Epoch 5/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2667 - accuracy: 0.8988 - val_loss: 0.3691 - val_accuracy: 0.8774
Epoch 6/10
782/782 [==============================] - 15s 20ms/step - loss: 0.2310 - accuracy: 0.9161 - val_loss: 0.3748 - val_accuracy: 0.8785
Epoch 7/10
782/782 [==============================] - 14s 18ms/step - loss: 0.2085 - accuracy: 0.9254 - val_loss: 0.3900 - val_accuracy:

#### As we can see introduction of 2-grams and 3-grams did not help. It may be due to the fact, that for all experiments the same number of 20000 tokens was used

### Using tf-idf output mode instead of multi-hot encoding

In [ ]:
# Create another TextVectorizer
tv4 = tf.keras.layers.TextVectorization(max_tokens = 20000,
                                        output_mode = 'tf-idf',
                                        ngrams = 1)

# Adapt TV to training dataset
tv4.adapt(train_data_text_only)

# Prepare preprocessed versions of all datasets
train_data_preprocessed_tfidf = train_data.map(lambda x, y: (tv4(x), y), num_parallel_calls = 4)
val_data_preprocessed_tfidf = val_data.map(lambda x, y: (tv4(x), y), num_parallel_calls = 4)
test_data_preprocessed_tfidf = test_data.map(lambda x, y: (tv4(x), y), num_parallel_calls = 4)

In [ ]:
# Check if everything OK
for inputs, targets in train_data_preprocessed_tfidf:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([19.388176   6.2761817  1.4221123 ...  0.         0.         0.       ], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
# Build and train model
model4 = build_model()

history4 = model.fit(train_data_preprocessed_tfidf,
          epochs = 10,
          validation_data = val_data_preprocessed_tfidf)

Epoch 1/10
782/782 [==============================] - 13s 17ms/step - loss: 0.4463 - accuracy: 0.8812 - val_loss: 0.7639 - val_accuracy: 0.8045
Epoch 2/10
782/782 [==============================] - 12s 15ms/step - loss: 0.2401 - accuracy: 0.9191 - val_loss: 0.8411 - val_accuracy: 0.8278
Epoch 3/10
782/782 [==============================] - 15s 19ms/step - loss: 0.1973 - accuracy: 0.9346 - val_loss: 0.8434 - val_accuracy: 0.8375
Epoch 4/10
782/782 [==============================] - 13s 16ms/step - loss: 0.1618 - accuracy: 0.9409 - val_loss: 0.8106 - val_accuracy: 0.8422
Epoch 5/10
782/782 [==============================] - 13s 16ms/step - loss: 0.1479 - accuracy: 0.9451 - val_loss: 0.8962 - val_accuracy: 0.8483
Epoch 6/10
782/782 [==============================] - 12s 15ms/step - loss: 0.1377 - accuracy: 0.9501 - val_loss: 0.9066 - val_accuracy: 0.8500
Epoch 7/10
782/782 [==============================] - 12s 15ms/step - loss: 0.1217 - accuracy: 0.9550 - val_loss: 0.9450 - val_accuracy:

#### We obtain ~85% accuracy

# Second attempt: sequence models

To implement sequence model, one has to do the following steps:
* represent samples as integer sequences (one integer for one word),
* map integer to a vector to obtain vector sequences,
* feed these vector sequences into 1-D CNN or RNN stack of layers

### Define TextVectorizer which for each sentence outputs sequence of integers

In [8]:
# Truncate each review to contain only 600 characters
tv5 = tf.keras.layers.TextVectorization(max_tokens = 20000,
                                        output_mode = 'int',
                                        output_sequence_length = 600)

# Adapting TextVectorizer
tv5.adapt(train_data_text_only)

# Transforming train, validation and test datasets
train_data_preprocessed_int = train_data.map(lambda x, y: (tv5(x), y), num_parallel_calls = 4)
val_data_preprocessed_int = val_data.map(lambda x, y: (tv5(x), y), num_parallel_calls = 4)
test_data_preprocessed_int = test_data.map(lambda x, y: (tv5(x), y), num_parallel_calls = 4)

# Check if everything OK
for inputs, targets in train_data_preprocessed_int:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32, 600)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(
[ 4897  6348  1138     2     1     1     5     4   774   753  6348   141
     4  2683   483     2  4897     3    32     2  2411  7979    21     2
    81   506     5     2     1    13   252   108    11   204 13217    16
    30    86   164  2124    51     9    86   366    46    10   153   118
    87    50    11   204    59    63    28   302   148   101    10    67
   226   108    34   383    56   165  1328    14    12     2   204    14
    85    19    56  1328    14   234    36     2   872  4897  6348     7
    52    71    41     4   588   968   204     9     7    29     5     2
    88    74    91   218   206   611  1120   968   122  1115     3   134
    80     9  1141    21    34   473 17470    13    11   204  1116   234
  9468    12     5     2    18     9    14   448    21     3    10   103
     9     7     4    53    50   454    12   700    15     4  3434 

### Building RNN model

In [ ]:
inputs = tf.keras.layers.Input(shape = (None, ), dtype = 'int64') # one input is sequence of integers
# Hot-encode each integer
encoded = tf.one_hot(inputs, depth = 20000)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(encoded)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.models.Model(inputs, outputs)

model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

history5 = model.fit(train_data_preprocessed_int,
          epochs = 10,
          validation_data = val_data_preprocessed_int)

#### The above model is very slow, since each review is encoded as 600 x 20000 matrix, which has 12000000 elements. Better encoding would be necessary, such as word embedding

### Same model as above but with embedding layer

In [ ]:
inputs = tf.keras.layers.Input(shape = (600, ))
x = tf.keras.layers.Embedding(input_dim = 20000, output_dim = 256, mask_zero = True)(inputs) # Notice mask_zero parameter
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(x)
x = tf.keras.layers.Dropout(rate = 0.5)(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

history6 = model.fit(train_data_preprocessed_int,
          epochs = 10,
          validation_data = val_data_preprocessed_int)

The model with embedding layer is learns a lot faster, but still not fast and accurately enough to beat simple feed-forward model. Let's try transformers.

### Build transformer encoder

In [9]:
class TransformerEncoder(tf.keras.layers.Layer):
  def __init__(self, num_heads, embed_dim, hidden_units, **kwargs):
    super().__init__(**kwargs)
    # Assign attributes
    self.num_heads = num_heads
    self.embed_dim = embed_dim
    self.hidden_units = hidden_units

    # Define all model's layers
    self.mha = tf.keras.layers.MultiHeadAttention(num_heads = self.num_heads, key_dim = self.embed_dim)
    self.dense = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units = self.hidden_units, activation = 'relu'),
        tf.keras.layers.Dense(units = self.embed_dim)
    ])
    # Normalization layer normalizes each sequence independently, opposite to BatchNormalization
    self.layer_normalization1 = tf.keras.layers.LayerNormalization()
    self.layer_normalization2 = tf.keras.layers.LayerNormalization()

  # Define forward propagation
  def call(self, inputs, mask = None):
    if mask is not None:
      # Since mask generated by embedding layer will be 2D we must expand its dims to be 3D (MHA expects 3D or 4D inputs)
      mask = mask[:, tf.newaxis, :]
    attention_output = self.mha(inputs, inputs, attention_mask = mask)
    proj_input = self.layer_normalization1(inputs + attention_output)
    proj_output = self.dense(proj_input)
    outputs = self.layer_normalization2(proj_input + proj_output)
    return outputs

  # Define get_config method, to be able to save and load model using this layer
  def get_config(self):
    config = super().get_config()
    config.update({
                  "embed_dim": self.embed_dim,
                  "num_heads": self.num_heads,
                  "dense_dim": self.hidden_units,
                  })
    return config


### Build PositionalEncoding layer to take into account word positions in sequence

In [11]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)
    # Initialize attributes
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim
    # Define embedding layer for inputs (tokens)
    self.token_embeddings = tf.keras.layers.Embedding(input_dim = self.input_dim,
                                                      output_dim = self.output_dim)
    # Define embedding for positions of tokens in the sequence
    self.position_embeddings = tf.keras.layers.Embedding(input_dim = self.sequence_length,
                                                         output_dim = self.output_dim)

  def call(self, inputs):
    # Get sequence length
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0, limit=length, delta=1)
    # Create embedding for tokens
    embedded_tokens = self.token_embeddings(inputs)
    # Create embedding for token positions
    embedded_positions = self.position_embeddings(positions)
    # Add and return both embeddings
    return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
      return tf.math.not_equal(inputs, 0)

    # Define get_config method, to be able to save and load model using this layer
    def get_config(self):
      config = super().get_config()
      config.update({
                    "output_dim": self.output_dim,
                    "sequence_length": self.sequence_length,
                    "input_dim": self.input_dim,
                    })
      return config


### Train Transformer model with Positional Embedding layer

In [ ]:
inputs = tf.keras.layers.Input(shape = (600, ), dtype = 'int64')
x = PositionalEmbedding(input_dim = 20000, output_dim = 256, sequence_length = 600)(inputs)
x = TransformerEncoder(num_heads = 2, embed_dim = 256, hidden_units = 32)(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)
x = tf.keras.layers.Dropout(rate = 0.5)(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

history7 = model.fit(train_data_preprocessed_int,
          epochs = 10,
          validation_data = val_data_preprocessed_int)